# Make word cloud with Kakao chat history

In [8]:
import pandas as pd
from os import path
import jieba, codecs, sys
from PIL import Image
from os import listdir
from os.path import isfile, join
from wordcloud import WordCloud, ImageColorGenerator
from scipy.misc import imread
import numpy as np
from collections import Counter, OrderedDict
import matplotlib as plt
from scipy.misc import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
class KakaoWordCloud():
    def __init__(self):
        self.text_path = ''
        self.background_picture_filename = ''
        self.f_name = '/Users/db91057/Library/Fonts/custom/NanumBarunGothic.ttf'
        
    def get_text(self):
        kakao_file_name = input("What is the file name: ")
        try:
            self.text_path = '../data/'+ kakao_file_name
            self.raw_text_df = pd.read_csv(self.text_path)
            print("Thank you for uploading your kakao chat history.")
        except ValueError:
            print("Oops! There is a problem with uploading your text. Try again...")
            
    def parse_message(self, text_df):
        message_list = text_df["Message"].values
        parse_words = [j for i in [i.split() for i in message_list] for j in i] 
        word_freq = Counter(parse_words).most_common()
        word_freq_dict = OrderedDict(word_freq)
        
        # Remove certain keys from the dict
        remove_list = ["Calling...","Video", "Photo","i","u","the",
                       "a","on","in","to","and","it","is","are","am",
                       "were","I","was","at","its","be","iam","have","has",
                       "had","for","this","that","of","do","or","got","as"]
        for f in remove_list:
            if word_freq_dict.get(f): word_freq_dict.pop(f)
        return word_freq_dict
    
    def divide_dataset(self):
        ids = list(set(self.raw_text_df["User"].tolist()))
        self.my_name = input("Which one is your chat ID? %s"%(ids))
        
        if self.my_name == ids[0]:
            self.friend_name = ids[1]
        else:
            self.friend_name = ids[0]
            
        self.my_text = self.raw_text_df[["User","Message"]].loc[self.raw_text_df["User"] == self.my_name]
        self.friend_text = self.raw_text_df[["User","Message"]].loc[self.raw_text_df["User"] == self.friend_name]
        
        self.my_text_dict = self.parse_message(self.my_text)
        self.friend_text_dict = self.parse_message(self.friend_text)
    
    def make_cloud(self, save = 1):
        for file in listdir("../data/templates/"):
            if file[-4:] != '.png' and file[-4:] != '.jpg':
                continue
            self.background_picture_filename = join("../data/templates/", file)

        bimg=imread(self.background_picture_filename)
        wordcloud=WordCloud(font_path=self.f_name, max_font_size=600,random_state=100,background_color='white',mask = bimg)
        
        print("My text to friend:")
        wordcloud=wordcloud.fit_words(self.my_text_dict)
        bimgColors=ImageColorGenerator(bimg)
        wordcloud.recolor(color_func=bimgColors)
        if save == 1:
            file_name = '../output/'+self.my_name + ".png"
            wordcloud.to_file(file_name)

        # show
        plt.figure(figsize=(8,8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        
        #########################################
        print("Friend's text to me:")
        wordcloud=wordcloud.fit_words(self.friend_text_dict)
        
        bimgColors=ImageColorGenerator(bimg)
        wordcloud.recolor(color_func=bimgColors)
        
        if save == 1:
            file_name = '../output/'+self.friend_name + ".png"
            wordcloud.to_file(file_name)
        
        # show
        plt.figure(figsize=(8,8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()


In [ ]:
Make = KakaoWordCloud()
Make.get_text()
Make.divide_dataset()
Make.make_cloud(1)